In [27]:
import os
import xml.etree.ElementTree as ET
import json
import re
import datetime
import exifread

In [28]:
gpx_file_names = [x for x in os.listdir('../assets/gps_tracks') if x.endswith('gpx')]
#gpx_file_names = ['d10_01_ride.gpx']
gpx_files = [os.path.join('../assets/gps_tracks', x) for x in gpx_file_names]
gpx_file_names[0]

'd01_01_ride.gpx'

In [29]:
all_coordinates = []
properties = []

one_list_coords = []
one_list_times = []

def append_to_properties(properties, day, section, category):
    properties.append({
        "day": day,
        "section": section,
        "category": category
    })

previous_day = 0
for gpx_file, gpx_file_name in zip(gpx_files, gpx_file_names):
    parts = gpx_file_name.split('_')
    day = int(parts[0][1:])
    if day != previous_day:
        section = 1
    previous_day = day
    exercise_type = parts[2].split('.')[0]
    
    
    tree = ET.parse(gpx_file)
    root = tree.getroot()

    trk = [c for c in root if c.tag.endswith('trk')]
    trksegs = [t for t in trk[0] if t.tag.endswith('trkseg')]
    trkpts = []

    for trkseg in trksegs:
        trkpts.extend([p for p in trkseg if p.tag.endswith('trkpt')])
    coordinates = [(float(t.attrib['lon']), float(t.attrib['lat'])) for t in trkpts]
    times = [datetime.datetime.strptime([t for t in x if t.tag.endswith('time')][0].text, "%Y-%m-%dT%H:%M:%SZ") for x in trkpts]
    one_list_coords.extend(coordinates)
    one_list_times.extend(times)
    
    last_idx = 0
    for c0, c1, i in zip(coordinates[:-1], coordinates[1:], range(len(coordinates)-1)):
        d2 = (c1[1] - c0[1])**2 + (c1[0] - c0[0])**2
        if d2 > 0.03**2:
            all_coordinates.append(coordinates[last_idx:i+1])
            last_idx = i+1
            append_to_properties(properties, day, section, exercise_type)
            section += 1
    
    if last_idx < len(coordinates):
        all_coordinates.append(coordinates[last_idx:])
        append_to_properties(properties, day, section, exercise_type)
        section += 1


## Photo time alignment

In [30]:
photo_file_names = [x for x in os.listdir('../assets/images/Scotland2018') if x.endswith('.JPG') and not x.endswith('Thumb.JPG')]
photo_files = [os.path.join('../assets/images/Scotland2018', x) for x in photo_file_names]
photo_file_names[0]

'P1020191.JPG'

In [31]:
photo_properties = {}
photo_times = {}
for i, file_path, file_name in zip(range(len(photo_files)), photo_files, photo_file_names):

    with open(file_path, 'rb') as f:
        # Return Exif tags
        tags = exifread.process_file(f)

    photo_times[file_name] = datetime.datetime.strptime(tags['EXIF DateTimeOriginal'].values, "%Y:%m:%d %H:%M:%S")
    photo_properties[file_name] = {
        'time_taken': datetime.datetime.strftime(photo_times[file_name], "%a %d %b %Y %H:%M"),
        'file_name': file_name,
        'id': i
    }

photo_times

{'P1020191.JPG': datetime.datetime(2018, 7, 16, 11, 34),
 'P1020192.JPG': datetime.datetime(2018, 7, 16, 12, 11, 59),
 'P1020193.JPG': datetime.datetime(2018, 7, 16, 12, 43, 58),
 'P1020194.JPG': datetime.datetime(2018, 7, 16, 12, 44, 14),
 'P1020195.JPG': datetime.datetime(2018, 7, 16, 12, 48, 29),
 'P1020196.JPG': datetime.datetime(2018, 7, 16, 12, 48, 48),
 'P1020197.JPG': datetime.datetime(2018, 7, 16, 14, 46, 35),
 'P1020198.JPG': datetime.datetime(2018, 7, 16, 15, 3, 59),
 'P1020199.JPG': datetime.datetime(2018, 7, 16, 15, 4, 5),
 'P1020200.JPG': datetime.datetime(2018, 7, 16, 15, 26, 44),
 'P1020201.JPG': datetime.datetime(2018, 7, 16, 17, 2, 51),
 'P1020202.JPG': datetime.datetime(2018, 7, 16, 17, 2, 56),
 'P1020203.JPG': datetime.datetime(2018, 7, 17, 9, 42, 53),
 'P1020204.JPG': datetime.datetime(2018, 7, 17, 10, 50, 51),
 'P1020205.JPG': datetime.datetime(2018, 7, 17, 12, 12, 26),
 'P1020206.JPG': datetime.datetime(2018, 7, 17, 12, 31, 51),
 'P1020207.JPG': datetime.datetime

### Match to nearest location

In [32]:
geotags = {}
for file_name, date_taken in photo_times.items():
    min_idx = 0
    min_d = datetime.timedelta(days=400)
    for i,t in enumerate(one_list_times):
        d = abs(date_taken - t)
        if d < min_d:
            min_d = d
            min_idx = i
    geotags[file_name] = one_list_coords[min_idx]


In [33]:
geotags

{'P1020191.JPG': (-4.583887, 56.003128),
 'P1020192.JPG': (-4.619849, 56.020494),
 'P1020193.JPG': (-4.636852, 56.10197),
 'P1020194.JPG': (-4.636852, 56.10197),
 'P1020195.JPG': (-4.636852, 56.10197),
 'P1020196.JPG': (-4.636852, 56.10197),
 'P1020197.JPG': (-4.709587, 56.202661),
 'P1020198.JPG': (-4.709587, 56.202661),
 'P1020199.JPG': (-4.709587, 56.202661),
 'P1020200.JPG': (-4.709587, 56.202661),
 'P1020201.JPG': (-4.536336, 56.209051),
 'P1020202.JPG': (-4.536336, 56.209051),
 'P1020203.JPG': (-4.586909, 56.249542),
 'P1020204.JPG': (-4.51279, 56.259658),
 'P1020205.JPG': (-4.428034, 56.227523),
 'P1020206.JPG': (-4.428034, 56.227523),
 'P1020207.JPG': (-4.428159, 56.227504),
 'P1020208.JPG': (-4.428159, 56.227504),
 'P1020209.JPG': (-4.428159, 56.227504),
 'P1020210.JPG': (-4.428159, 56.227504),
 'P1020211.JPG': (-4.442399, 56.346365),
 'P1020212.JPG': (-4.291234, 56.380274),
 'P1020213.JPG': (-4.2923, 56.484988),
 'P1020214.JPG': (-4.291251, 56.485002),
 'P1020216.JPG': (-4.26

In [34]:
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": c
            },
            "properties": p
        } for c,p in zip(all_coordinates, properties)]
}

geojson['features'].extend([
    {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": c
        },
        "properties": photo_properties[file_name]
    } for file_name, c in geotags.items()])

## Write geojson string and remove quotes from numbers

In [35]:
geojson_string = json.dumps(geojson)

new_string = re.sub(r'"(-?\d+(.\d+)?)"', lambda x: x.group(1), geojson_string)
new_string[:2000]

'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "LineString", "coordinates": [[-1.121485, 52.37534], [-1.12135, 52.375283], [-1.121242, 52.375179], [-1.121699, 52.374698], [-1.121975, 52.374476], [-1.122574, 52.37406], [-1.122717, 52.373991], [-1.122841, 52.374], [-1.122974, 52.374048], [-1.123733, 52.374372], [-1.124551, 52.374709], [-1.124903, 52.374847], [-1.125696, 52.375096], [-1.125952, 52.375169], [-1.126611, 52.375347], [-1.127479, 52.375538], [-1.127758, 52.375568], [-1.127845, 52.375585], [-1.128019, 52.375631], [-1.12857, 52.375922], [-1.128654, 52.376006], [-1.128709, 52.376095], [-1.128761, 52.376286], [-1.128761, 52.376286], [-1.128761, 52.376286], [-1.129033, 52.37838], [-1.129179, 52.378598], [-1.129325, 52.378816], [-1.129618, 52.379251], [-1.129643, 52.379363], [-1.129669, 52.379474], [-1.129694, 52.379586], [-1.12972, 52.379697], [-1.129874, 52.380365], [-1.130051, 52.380477], [-1.13028, 52.380515], [-1.130508, 52.380554], [-1.130

In [36]:
with open('../assets/gps_tracks/all_tracks.json', 'w') as f:
    f.write(new_string)